<a href="https://colab.research.google.com/github/SergIvanov59/think_live/blob/main/PROJECT_4_IVANOV_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ЦЕЛЬ
Создать простую рекомендательную систему, которая будет подсказывать пользователям, какие курсы могут быть им интересны. Такая система позволит организовать допродажу в корзине и тем самым увеличить средний чек.
ЗАДАЧИ
1. Разработать таблицу, в которой каждому курсу будет соответствовать по две рекомендации.
2. Сформулировать комментарии, которые бы разъясняли, что где и почему вы делаете.
ФОРМАЛИЗОВАННЫЕ ЗАДАЧИ
1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2. Обработать данные средствами Python.
3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.

1-й этап работы
1. Изучаем содержимое двух рабочих таблиц продажи курсов.
2. Готовим данные для анализа. Для этого при помощи SQL-запроса выгружаем данные по всем продажам в разрезе пользователей.
3. Обрабатываем данные. Разбиваем покупки курсов по парам и ранжируем для каждого курса.

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras

In [ ]:
# Посчитаем сколько клиентов покупали курсы
def Clients_Course():
    query = '''select count(distinct user_id)
from final.cart_items i join final.carts c on i.cart_id=c.id
where state='successful' and resource_type='Course'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
course_buyers = pd.DataFrame(Clients_Course())
course_buyers

,count
0,49006


In [ ]:
# Посчитаем сколько различных курсов в продаже
def Clients_Course():
    query = '''select count(distinct resource_id)
from final.cart_items
where resource_type='Course'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
all_courses = pd.DataFrame(Clients_Course())
all_courses

,count
0,127


In [ ]:
# Посчитаем среднее число купленных курсов на одного клиента
def Clients_Course():
    query = '''with clients as
(
select user_id, count(resource_type) as course_count
from final.cart_items i join final.carts c on i.cart_id=c.id
where state='successful' and resource_type='Course'
group by 1
)
select avg(course_count)
from clients
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
avg_courses_1buyer = pd.DataFrame(Clients_Course())
avg_courses_1buyer

,avg
0,1.4402522140146105


In [ ]:
# Посчитаем сколько клиентов купили больше одного курса
def Clients_Course():
    query = '''with clients as
(
select user_id, count(distinct resource_id) as course_count
from final.cart_items i join final.carts c on i.cart_id=c.id
where state='successful' and resource_type='Course'
group by 1
)
select count(user_id)
from clients
where course_count > 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
clients_course_more_1 = pd.DataFrame(Clients_Course())
clients_course_more_1

,count
0,12656


In [ ]:
# Создаем DF с данными о продажах пользователям, купившим больше 1 курса.
def Clients_Course():
    query = '''with clients as
(
select user_id, count(distinct resource_id) as course_count
from final.cart_items i join final.carts c on i.cart_id=c.id
where state='successful' and resource_type='Course'
group by 1
)
select clients.user_id, i.resource_id
from clients join final.carts c on clients.user_id=c.user_id join final.cart_items i on i.cart_id=c.id
where clients.course_count > 1 and c.state='successful' and i.resource_type='Course'
order by clients.user_id, i.resource_id
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
df = pd.DataFrame(Clients_Course())
df.head()

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


2-й этап работы
Создаем итоговую таблицу с рекомендациями
За основу возьмём информацию о клиентах, купивших больше одного курса.
Их предпочтения - это наши рекмендации новым клиентам.
1. Создадим столбец со списками уникальных курсов для каждого клиента
2. В работу берём покупки больше одного курса. Разбиваем все покупки курсов по парам.
3. Сортируем по популярности пары курсов. Нас интересуют самые популярные пары курсов.
4. Отсортировываем пары по id первой покупки (первый курс) и выбираем вторые id покупок (второй курс) в первых двух самых популярных парах.
5. Вместо малопопулярных курсов предложим пару самых популярных. 

In [ ]:
# Создадим столбец со списками уникальных курсов для каждого клиента
# Каждому пользователю будет соответствовать список id курсов, которые он приобрёл
course_df = df.groupby(['user_id'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
course_df = course_df.rename(columns={'resource_id':'courses'})
course_df

,user_id,courses
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


In [ ]:
# При помощи функции itertools.combinations() создаем кортеж пар курсов, в котором нет повторяющихся элементов.
import itertools
courses_list = list()
for courses in course_df['courses']:
    for course_pair in itertools.combinations(courses,2):
        courses_list.append(course_pair)
courses_list
print("Всего пар курсов:",len(courses_list))
print("Всего различных пар курсов:",len(set(courses_list)))

Всего пар курсов: 40017
Всего различных пар курсов: 3989


In [ ]:
# Методом Counter создадим коллекцию самых популярных пар курсов. 
from collections import Counter
popular = Counter(courses_list)
print(popular)

Counter({(551, 566): 797, (515, 551): 417, (489, 551): 311, (523, 551): 304, (566, 794): 290, (489, 515): 286, (490, 566): 253, (490, 551): 247, (570, 752): 247, (569, 572): 216, (515, 523): 213, (553, 745): 212, (489, 523): 206, (569, 840): 204, (514, 551): 200, (516, 745): 199, (515, 566): 195, (489, 566): 188, (504, 572): 184, (572, 840): 178, (551, 552): 177, (507, 570): 172, (490, 809): 163, (489, 490): 152, (507, 752): 150, (523, 552): 144, (490, 515): 143, (551, 570): 142, (504, 569): 139, (514, 515): 139, (551, 745): 138, (514, 566): 138, (502, 551): 135, (504, 840): 135, (571, 1125): 122, (502, 566): 120, (523, 566): 120, (570, 809): 119, (752, 809): 115, (490, 523): 114, (357, 571): 112, (523, 564): 110, (551, 749): 109, (516, 553): 107, (551, 777): 107, (551, 679): 104, (356, 571): 103, (551, 564): 103, (515, 749): 103, (568, 745): 102, (356, 357): 100, (363, 511): 99, (551, 571): 98, (551, 809): 96, (502, 514): 95, (551, 794): 95, (490, 514): 94, (566, 764): 92, (490, 564):

In [ ]:
# Опредилим самую популярную пару курсов.
Counter(courses_list).most_common(1)[0][0]

(551, 566)

In [ ]:
# Создадим Датафрейм популярных курсов. Count - это сколько раз встречается данная пара курсов. 
popular_df = pd.DataFrame(Counter(courses_list).most_common(),columns=['pairs','count'])
popular_df

,pairs,count
0,"(551, 566)",797
1,"(515, 551)",417
2,"(489, 551)",311
3,"(523, 551)",304
4,"(566, 794)",290
...,...,...
3984,"(741, 791)",1
3985,"(361, 1128)",1
3986,"(743, 1156)",1
3987,"(864, 1129)",1


In [ ]:
# Первые две пары показывают, что логично рассматривать в качестве рекомендаций и обратное сочетание пар. 
# Т.е. для курса 551 лучшая рекомендация - это курсы 566 и 515
# Разобьём пары курсов на два столбца с resource_id курсов.
# Столбец course - основной курс, столбец course_rec - рекомендованный курс.
# Создадим два датафрейма. Прямое сочетание пар курсов и обратное.
df_1 = popular_df.copy()
df_2 = popular_df.copy()

df_1['course'] = df_1['pairs'].apply(lambda x: x[0])
df_1['course_rec'] = df_1['pairs'].apply(lambda x: x[1])

df_2['course'] = df_2['pairs'].apply(lambda x: x[1])
df_2['course_rec'] = df_2['pairs'].apply(lambda x: x[0])

df_1,df_2

(            pairs  count  course  course_rec
 0      (551, 566)    797     551         566
 1      (515, 551)    417     515         551
 2      (489, 551)    311     489         551
 3      (523, 551)    304     523         551
 4      (566, 794)    290     566         794
 ...           ...    ...     ...         ...
 3984   (741, 791)      1     741         791
 3985  (361, 1128)      1     361        1128
 3986  (743, 1156)      1     743        1156
 3987  (864, 1129)      1     864        1129
 3988  (756, 1185)      1     756        1185
 
 [3989 rows x 4 columns],
             pairs  count  course  course_rec
 0      (551, 566)    797     566         551
 1      (515, 551)    417     551         515
 2      (489, 551)    311     551         489
 3      (523, 551)    304     551         523
 4      (566, 794)    290     794         566
 ...           ...    ...     ...         ...
 3984   (741, 791)      1     791         741
 3985  (361, 1128)      1    1128         361
 3986 

In [ ]:
# Объединим датафреймы
df_common = pd.concat([df_1, df_2]).reset_index(drop=True)
df_common

,pairs,count,course,course_rec
0,"(551, 566)",797,551,566
1,"(515, 551)",417,515,551
2,"(489, 551)",311,489,551
3,"(523, 551)",304,523,551
4,"(566, 794)",290,566,794
...,...,...,...,...
7973,"(741, 791)",1,791,741
7974,"(361, 1128)",1,1128,361
7975,"(743, 1156)",1,1156,743
7976,"(864, 1129)",1,1129,864


In [ ]:
# Добавим столбец ранжирования пар для каждого курса. 
# Нас будут интересовать у каждого курса из course курсы из столбца course_rec с rank 1 и 2.
df_common['rank'] = df_common.sort_values(by='count', ascending=False).groupby('course').cumcount()+1

In [ ]:
# Посмотри как выглядит наша таблица для одного конкретного курса 551
df_common[df_common['course'] == 551].sort_values('rank')

,pairs,count,course,course_rec,rank
0,"(551, 566)",797,551,566,1
3990,"(515, 551)",417,551,515,2
3991,"(489, 551)",311,551,489,3
3992,"(523, 551)",304,551,523,4
3996,"(490, 551)",247,551,490,5
...,...,...,...,...,...
6918,"(360, 551)",1,551,360,113
3204,"(551, 1182)",1,551,1182,114
3170,"(551, 866)",1,551,866,115
3824,"(551, 830)",1,551,830,116


In [ ]:
# Оставим только записи соответствующие первому и второму рангу, удалим столбец 'pairs'
df_common = df_common[(df_common['rank'] == 1) | (df_common['rank'] == 2)]
df_common.drop('pairs', axis=1, inplace=True)

C:\Users\mvideo\AppData\Local\Temp\ipykernel_10624\4190723474.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_common.drop('pairs', axis=1, inplace=True)


In [ ]:
# Посмотрим как это будет выглядеть для конкретного курса 551
df_common[df_common['course'] == 551]

,count,course,course_rec,rank
0,797,551,566,1
3990,417,551,515,2


In [ ]:
# Оставим одну строку для каждого курса, а вторую по популярности запишем рядом
# Переставим столбцы для удобства восприятия
df_rec = df_common[df_common['rank'] == 1].merge(df_common[df_common['rank'] == 2], on='course', how='left', suffixes=('_1', '_2'))
df_rec = df_rec.reindex(columns=['course', 'course_rec_1', 'count_1', 'rank_1', 'course_rec_2', 'count_2', 'rank_2'])
df_rec

,course,course_rec_1,count_1,rank_1,course_rec_2,count_2,rank_2
0,551,566,797,1,515,417,2
1,515,551,417,1,489,286,2
2,489,551,311,1,515,286,2
3,523,551,304,1,515,213,2
4,490,566,253,1,551,247,2
...,...,...,...,...,...,...,...
121,833,507,2,1,570,2,2
122,1201,571,1,1,1144,1,2
123,1182,553,1,1,1103,1,2
124,902,519,1,1,777,1,2


In [ ]:
# Некоторые пары курсов имеют низкую частоту совместных продаж, можно предположить, что эти курсы были проданы вместе случайно 
# и нет смысла  рекомендовать их в паре друг с другом в будущем.
# Необходимо установить минимальную границу — какое количество раз считать слишком малым.
# Для этого посмотрим статистическое распределение числа продаж.
df_rec['count_1'].describe()

count    126.000000
mean      79.436508
std      120.688458
min        1.000000
25%       16.250000
50%       37.500000
75%       97.250000
max      797.000000
Name: count_1, dtype: float64

In [ ]:
# Нижняя квартиль Q1 составляем 16 продаж. Ниже этой цифры число продаж слишком малое. 
# Возьмём эту цифру за минимальную величину, при значениях ниже которой будем предлагать наиболее популярную пару.
df_rec.loc[df_rec['count_1'] < 16, 'course_rec_1'] = 551
df_rec.loc[df_rec['count_2'] < 16, 'course_rec_2'] = 566
df_rec

,course,course_rec_1,count_1,rank_1,course_rec_2,count_2,rank_2
0,551,566,797,1,515,417,2
1,515,551,417,1,489,286,2
2,489,551,311,1,515,286,2
3,523,551,304,1,515,213,2
4,490,566,253,1,551,247,2
...,...,...,...,...,...,...,...
121,833,551,2,1,566,2,2
122,1201,551,1,1,566,1,2
123,1182,551,1,1,566,1,2
124,902,551,1,1,566,1,2


In [ ]:
# Удалим вспомогательные столбцы и приведем таблицу к окончательному виду.
# В ней столбец course - это курс, который пользователь выбрал в корзину,
# А в столбцах course_rec_1 и course_rec_2 - рекомендуемые ему курсы.
df_rec.drop(['count_1', 'rank_1', 'count_2', 'rank_2'], axis=1, inplace=True)
df_rec = df_rec.sort_values(by='course')
df_rec

,course,course_rec_1,course_rec_2
15,356,571,357
14,357,571,356
29,358,570,752
33,359,570,566
32,360,745,566
...,...,...,...
102,1188,551,566
114,1198,551,566
119,1199,551,566
125,1200,551,566


ИТОГОВАЯ ТАБЛИЦА С РЕКОМЕНДАЦИЯМИ 
Идея таблицы.
 Каждый пользователь, покупающий курс психологически больше готов сделать новую покупку. И многие пользователи совершили вторую покупку. 
 По каким-то особенностям они предпочли именно этот курс для второй покупки, который находится в наиболее популярной паре. 
 Факт - пользователь, купивший курс с id 551 в 797 случаях купил и курс id 566. 
 Логично это использовать для увеличения продаж. 
 Логично также купившему курс с id 566 купить и курс id 551. 
 То есть, для формирования рекомендаций используем прямое и обратное сочетание самых популярных пар курсов.
 В нижней квартили Q1 статистического распределения продаж курсов слишком мало и возможно эти курсы были проданы вместе случайно.
 Вместо этих малопопулярных курсов мы предложим пользователю пару самых популярных курсов 551 и 566.
 Обратим их внимание на курсы, которые понравились достаточно большому числу пользователей.
 Предлагаемая таблица будет рекомендовать клиентам курсы, которые в большей степени вероятности должны их заинтересовать.